# Imports

In [1]:
import numpy as np
import pandas as pd

from datetime import datetime

import twint

# Fixes runtime errors with twint
import nest_asyncio
nest_asyncio.apply()

from textblob import TextBlob

import matplotlib
%matplotlib inline

# Scrape Tweets

In [2]:
def twint_search(search, username=None, since=None, until=None, drop_cols=None, limit=None):
    '''
    Function to return a pandas dataframe of tweets in English containing search terms using twint.
    Required parameter: search term.
    Optional parameters: username, start date (since) and end date (until) to search, columns to drop, maximum number of tweets (limit).
    '''
    c = twint.Config()
    c.Lang = 'en'
    c.Search = search
    c.Username = username
    c.Since = since
    c.Until = until
    c.Limit = limit
    c.Pandas = True
    # Hide the printing of every tweet during scrape
    c.Hide_output = True
    twint.run.Search(c)
    df = twint.storage.panda.Tweets_df
    # Transform date string into datetime object
    df['date'] = pd.to_datetime(df['date']).dt.date
    return df

In [3]:
def search_loop(start_date, end_date, search, filename, username=None, drop_cols=None, limit=None):
    '''
    Function to loop over date range and perform twint_search function for each day, returning one combined dataframe.
    Periodically saves progress to CSV after each daily search.
    Required parameters: start date, end date, search term.
    Optional parameters: username, columns to drop, maximum number of tweets per day (limit).
    '''
    df = pd.DataFrame()
    date_range = pd.Series(pd.date_range(start_date, end_date))
    for d in range(len(date_range) - 1):
        since = date_range[d].strftime('%Y-%m-%d')
        until = date_range[d + 1].strftime('%Y-%m-%d')
        day_df = twint_search(search=search, username=username, since=since, until=until, drop_cols=drop_cols, limit=limit)
        # Drop empty columns
        day_df.drop(columns=drop_cols, axis=1, inplace=True)
        # Add new daily data to dataframe, reset index, save to CSV
        df = pd.concat([df, day_df])
        del day_df
        df.reset_index(drop=True, inplace=True)
        df.to_csv(f'10k_Datasets/{filename}.csv')
        print(datetime.now(), f'{since} Saved!')
    return df

In [4]:
# Search loop params for March tweets

start_date = '2020-03-28'

end_date = '2020-03-29'

search = '(mask OR masks) AND (covid OR covid19 OR corona OR coronavirus)'

filename = '10k_march28_tweets'

drop_cols = ['timezone', 'place', 'cashtags', 'user_id_str', 'day', 'hour', 'search', 'near', 'geo',
             'source', 'user_rt_id', 'user_rt', 'retweet_date', 'translate', 'trans_src', 'trans_dest']

In [5]:
%time df = search_loop(start_date=start_date, end_date=end_date, search=search, filename=filename, drop_cols=drop_cols, limit=10000)

2020-06-15 21:05:58.350979 2020-03-28 Saved!
CPU times: user 52.8 s, sys: 338 ms, total: 53.2 s
Wall time: 2min 17s


In [6]:
len(df)

3441